In [63]:
import pandas as pd
import os
Drug = pd.read_csv(os.path.join("webmd.csv"))
Drug = pd.read_csv("webmd.csv")
Drug.head()

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,75 or over,Stuffy Nose,9/21/2014,25dph-7.5peh,146724,5,5,I'm a retired physician and of all the meds I ...,5,Male,"Drowsiness, dizziness , dry mouth /nose/thro...",0
1,25-34,Cold Symptoms,1/13/2011,25dph-7.5peh,146724,5,5,cleared me right up even with my throat hurtin...,5,Female,"Drowsiness, dizziness , dry mouth /nose/thro...",1
2,65-74,Other,7/16/2012,warfarin (bulk) 100 % powder,144731,2,3,why did my PTINR go from a normal of 2.5 to ov...,3,Female,,0
3,75 or over,Other,9/23/2010,warfarin (bulk) 100 % powder,144731,2,2,FALLING AND DON'T REALISE IT,1,Female,,0
4,35-44,Other,1/6/2009,warfarin (bulk) 100 % powder,144731,1,1,My grandfather was prescribed this medication ...,1,Male,,1


In [64]:
X = Drug[["EaseofUse", "Effectiveness","DrugId", "Sex"]]
y = Drug[["Satisfaction"]]
X.head()


,EaseofUse,Effectiveness,DrugId,Sex
0,5,5,146724,Male
1,5,5,146724,Female
2,2,3,144731,Female
3,2,2,144731,Female
4,1,1,144731,Male


In [65]:
len(X[X.isna().any(axis=1)])

from sklearn.impute import SimpleImputer
import numpy as np
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
si.fit(X)
X_ = si.transform(X)
X = pd.DataFrame(X_, columns=X.columns)
len(X[X.isna().any(axis=1)])

0

In [66]:
from sklearn import preprocessing
oe = preprocessing.OrdinalEncoder().fit(X[["Sex"]])
X[["Sex"]] = oe.transform(X[["Sex"]])

In [67]:
from sklearn import preprocessing
standard_scaler = preprocessing.StandardScaler().fit(X[["EaseofUse", "Effectiveness"]])

def scale(dataframe, columns, scaler):
    scaled = scaler.transform(dataframe[columns])
    dataframe[columns] = pd.DataFrame(scaled, columns=columns)
    return dataframe

X = scale(X, ["EaseofUse", "Effectiveness"], standard_scaler)
X.head()

,EaseofUse,Effectiveness,DrugId,Sex
0,0.743845,1.008228,146724,2.0
1,0.743845,1.008228,146724,1.0
2,-1.532833,-0.365437,144731,1.0
3,-1.532833,-1.052269,144731,1.0
4,-2.291725,-1.739102,144731,2.0


In [68]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size = 0.2, random_state = 42
)

len(X_train), len(X_test)

(290244, 72562)

In [69]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

algorithms = {
    "Decision Tree" : DecisionTreeClassifier(max_depth=5),
    "Nearest Neighbors" : KNeighborsClassifier(3),
    "Random Forest" : RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "Naive Bayes" : GaussianNB(),
    "LDA" : LinearDiscriminantAnalysis(),
}
for name, algorithm in algorithms.items():
    algorithm.fit(X_train, y_train.values.ravel())
    score = algorithm.score(X_test, y_test)
    print(name, round(score,2))

Decision Tree 0.67
Nearest Neighbors 0.59
Random Forest 0.63
Naive Bayes 0.53
LDA 0.61


In [70]:
import joblib
joblib.dump(algorithms["Decision Tree"], 'Drug_Decision_Tree.joblib')
best_model = joblib.load('Drug_Decision_Tree.joblib')
best_model

DecisionTreeClassifier(max_depth=5)

In [71]:
Test1 = pd.DataFrame([[5,5,146724,1]], columns=X_train.columns)
Test1 = scale(Test1, ["EaseofUse", "Effectiveness"], standard_scaler)
Test1.head()

Test2 = pd.DataFrame([[5,4,146724,0]], columns=X_train.columns)
Test2 = scale(Test2, ["EaseofUse", "Effectiveness"], standard_scaler)
Test2.head()
Ausgabe = pd.concat([Test1, Test2])
Ausgabe.head()

,EaseofUse,Effectiveness,DrugId,Sex
0,0.743845,1.008228,146724,1
0,0.743845,0.321396,146724,0


In [72]:
predictions = best_model.predict(pd.concat([Test1, Test2]))

for person, prediction in zip(["Test1", "Test2"], predictions):

    print(prediction)

5
4


In [73]:
from sklearn.metrics import classification_report
best_model.fit(X_train, y_train)
predictions = best_model.predict(X_test)
print(classification_report(predictions, y_test))


              precision    recall  f1-score   support

           1       0.73      0.82      0.77     17932
           2       0.13      0.39      0.19      2346
           3       0.49      0.47      0.48     11005
           4       0.64      0.52      0.58     15386
           5       0.87      0.75      0.81     25893
           6       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0

    accuracy                           0.67     72562
   macro avg       0.41      0.42      0.41     72562
weighted avg       0.71      0.67      0.68     72562



C:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression

# instantiate model
logreg = LogisticRegression()

# fit model
logreg.fit(X_train, y_train)

C:\Python310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [78]:
# make class predictions for the testing set
y_pred_class = logreg.predict(X_test)

In [79]:
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.2819933298420661


In [80]:
# examine the class distribution of the testing set (using a Pandas Series method)
y_test.value_counts()

Satisfaction
5               22334
1               20164
4               12518
3               10452
2                7092
6                   1
10                  1
dtype: int64

In [81]:
# calculate the percentage of ones
# because y_test only contains ones and zeros, we can simply calculate the mean = percentage of ones
y_test.mean()

Satisfaction    3.134726
dtype: float64

In [82]:
# calculate the percentage of zeros
1 - y_test.mean()

Satisfaction   -2.134726
dtype: float64

In [83]:
# calculate null accuracy in a single line of code
# only for binary classification problems coded as 0/1
max(y_test.mean(), 1 - y_test.mean())

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [84]:
# calculate null accuracy (for multi-class classification problems)
y_test.value_counts().head(1) / len(y_test)

Satisfaction
5               0.307792
dtype: float64

In [85]:
# print the first 25 true and predicted responses
print('True:', y_test.values[0:25])
print('False:', y_pred_class[0:25])

True: [[5]
 [4]
 [1]
 [1]
 [5]
 [5]
 [1]
 [2]
 [5]
 [3]
 [4]
 [4]
 [5]
 [5]
 [5]
 [4]
 [2]
 [5]
 [3]
 [5]
 [1]
 [3]
 [5]
 [3]
 [1]]
False: [1 1 1 1 1 1 1 1 1 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [86]:
# IMPORTANT: first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
print(metrics.confusion_matrix(y_test, y_pred_class))

[[20161     0     0     0     3     0     0]
 [ 7083     0     0     0     9     0     0]
 [10399     0     0     0    53     0     0]
 [12408     0     0     0   110     0     0]
 [22033     0     0     0   301     0     0]
 [    1     0     0     0     0     0     0]
 [    1     0     0     0     0     0     0]]


In [87]:
# print the first 25 true and predicted responses
print('True', y_test.values[0:25])
print('Pred', y_pred_class[0:25])

True [[5]
 [4]
 [1]
 [1]
 [5]
 [5]
 [1]
 [2]
 [5]
 [3]
 [4]
 [4]
 [5]
 [5]
 [5]
 [4]
 [2]
 [5]
 [3]
 [5]
 [1]
 [3]
 [5]
 [3]
 [1]]
Pred [1 1 1 1 1 1 1 1 1 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [88]:
# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred_class)
print(confusion)
#[row, column]
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

[[20161     0     0     0     3     0     0]
 [ 7083     0     0     0     9     0     0]
 [10399     0     0     0    53     0     0]
 [12408     0     0     0   110     0     0]
 [22033     0     0     0   301     0     0]
 [    1     0     0     0     0     0     0]
 [    1     0     0     0     0     0     0]]


In [89]:
# use float to perform true division, not integer division
print((TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred_class))

0.7400161503450301
0.2819933298420661


In [90]:
classification_error = (FP + FN) / float(TP + TN + FP + FN)

print(classification_error)
print(1 - metrics.accuracy_score(y_test, y_pred_class))

0.2599838496549699
0.718006670157934


In [96]:
sensitivity = TP / float(FN + TP)

print(sensitivity)
print(metrics.recall_score(y_test, y_pred_class, average="micro"))

0.0
0.2819933298420661


In [93]:
specificity = TN / (TN + FP)

print(specificity)

1.0


In [94]:
false_positive_rate = FP / float(TN + FP)

print(false_positive_rate)
print(1 - specificity)

0.0
0.0


In [97]:
precision = TP / float(TP + FP)

print(precision)
print(metrics.precision_score(y_test, y_pred_class, average="micro"))

nan
0.2819933298420661


C:\Users\Eileen Bréguet\AppData\Local\Temp\ipykernel_1232\282641192.py:1: RuntimeWarning: invalid value encountered in true_divide
  precision = TP / float(TP + FP)


Fazit: Mein Modell ist nicht wirklich dazu geeignet eine Wahrheitsmatrix zu erstellen. Ich habe keine Daten die ich mit 0 und 1 auswerten kann. Der eine Fehler ist wegen den Spalten, weil sie kein Boolean sind. Es macht von mir aus keinen Sinn sie umzu konvertieren. Ich musste noch bei den precision average= hinzufügen, dass es überhaupt geklappt hat.